In [15]:
naam = "Onno de Jong"
opdracht = "Inleveropdracht 2: Logistic Regression & Naïve Bayes"
deel = 1

In [1]:
import nltk
from nltk import TweetTokenizer
nltk.download("twitter_samples")
nltk.download("stopwords")
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer

from string import punctuation
import numpy as np

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\maste\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maste\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# imports vorige files

tokenizer = TweetTokenizer()
swords = stopwords.words("english")
stemmr = PorterStemmer()

def filtertxt(text, filtr):
    return nltk.re.sub(filtr, "", text)

def removewords(text, filtr):
    return [word for word in text if word not in filtr]

def clean(tweet, printsteps=False):
    print("origineel \t\t", tweet) if printsteps else None
    # rt's verwijderen, het is geen nuttige informatie dat we weten oof de tweet een retweet is voor ons doel
    tweet = filtertxt(tweet, "RT")
    print("Filter de RT's \t\t", tweet) if printsteps else None
    
    # #'s verwijderen, een # is in principe een leesteken maar met de tweettokenizer wordt de hele hashtag 
    # als 1 token gezien dus we moet hier al de #'s eruit halen, ook is het geen nuttige informatie
    tweet = filtertxt(tweet, "#")
    print("Filter de #'s' \t\t", tweet) if printsteps else None
     
    # tokenize de tweet
    tweet = tokenizer.tokenize(tweet)
    print("Tokenize de tweet \t", tweet) if printsteps else None
    
    # stopworden uit de tweet halen, geen nuttige informatie, deze worden geven niet aan of het een blije of een boze tweet is
    tweet = removewords(tweet, swords)
    print("verwijder stopworden \t", tweet) if printsteps else None
    
    # punctuatie uit de tweet halen, geen nuttige informatie, punctuatie geeft niet aan of het een blije of een boze tweet is
    tweet = removewords(tweet, punctuation)
    print("verwijder punctuatie \t", tweet) if printsteps else None
    
    # stemmer toepassen op de worden, omdat we zometeen de worden gaan tellen is het van groot belang dat de worden allemaal in dezelfde vorm staan,
    # bijvoorbeeld "invitation" en "invite" worden allebei "invit"
    tweet = list(map(stemmr.stem, tweet))
    print("stem de tweet \t\t", tweet) if printsteps else None
    
    # de tweet is helemaal opgeschoond en klaar voor gebruik
    return tweet

def counttweets(tweets, pos):
    freq = {}
    for tweet in tweets:
        for word in tweet:
            if (word, pos) in freq.keys():
                freq[(word, pos)] += 1
            else:
                freq[(word, pos)] = 1
    return freq

def getres(tweets):
    res = {}
    done = []
    for word, pos in tweets.keys():
        if word in done:
            continue
            
        done.append(word)
        # als het tegendeel niet bestaat is het 0
        neg_res = 0 if (word, 0.0) not in tweets.keys() else tweets[word, 0.0]
        pos_res = 0 if (word, 1.0) not in tweets.keys() else tweets[word, 1.0]
        res[word] = [pos_res, neg_res]
                
    return res

In [3]:
# trainset
trainset = twitter_samples.strings('positive_tweets.json')[:int(len(twitter_samples.strings('positive_tweets.json')) * .8)] \
                                       + twitter_samples.strings('negative_tweets.json')[:int(len(twitter_samples.strings('negative_tweets.json')) * .8)]

# validatieset
validatieset = twitter_samples.strings('positive_tweets.json')[int(len(twitter_samples.strings('positive_tweets.json')) * .8):] \
                                           + twitter_samples.strings('negative_tweets.json')[int(len(twitter_samples.strings('negative_tweets.json')) * .8):]

print(len(trainset), len(validatieset))

8000 2000


In [4]:
pos_freq_tweets = counttweets([clean(tweet) for tweet in twitter_samples.strings('positive_tweets.json')], 1.0)
neg_freq_tweets = counttweets([clean(tweet) for tweet in twitter_samples.strings('negative_tweets.json')], 0.0)

all_tweets = pos_freq_tweets | neg_freq_tweets

freqs = getres(all_tweets)
freqs

{'followfriday': [25, 0],
 '@france_int': [1, 0],
 '@pkuchly57': [1, 0],
 '@milipol_pari': [1, 0],
 'top': [32, 6],
 'engag': [7, 0],
 'member': [16, 6],
 'commun': [33, 2],
 'week': [83, 56],
 ':)': [3691, 2],
 '@lamb2ja': [1, 0],
 'hey': [77, 26],
 'jame': [7, 4],
 'how': [24, 23],
 'odd': [2, 3],
 ':/': [5, 11],
 'pleas': [99, 275],
 'call': [37, 29],
 'contact': [7, 7],
 'centr': [2, 2],
 '02392441234': [1, 0],
 'abl': [8, 23],
 'assist': [1, 0],
 'mani': [33, 29],
 'thank': [643, 107],
 '@despiteoffici': [1, 0],
 'listen': [17, 18],
 'last': [47, 73],
 'night': [68, 46],
 'as': [5, 9],
 'you': [123, 84],
 'bleed': [2, 1],
 'amaz': [51, 16],
 'track': [5, 3],
 'when': [14, 30],
 'scotland': [2, 2],
 '@97side': [1, 0],
 'congrat': [21, 2],
 'yeaaaah': [1, 0],
 'yippppi': [1, 0],
 'accnt': [2, 0],
 'verifi': [2, 0],
 'rqst': [1, 0],
 'succeed': [1, 0],
 'got': [69, 104],
 'blue': [9, 5],
 'tick': [1, 0],
 'mark': [1, 2],
 'fb': [6, 7],
 'profil': [2, 1],
 '15': [5, 10],
 'day': [246,

functie om de sum van alle pos/neg waardes uit de dict te halen 

In [5]:
def total(freq, position):
    tot = 0
    for value in freq.values():
        tot += value[position]
    return tot

In [6]:
def naive_bayes(freq, samples, lables):
    # de manier waarop mijn frequencie dict is opgebouwd zitten er alleen maar unieke values in de keys
    v = len(freq)
    
    # freqpos en freqneg zitten al in mijn frequenciedict
    # dit is al gedaan doormiddel van getres
    
    # aantal positieve en negatieve counts door middel van de total functie
    Npos, Nneg = total(freq, 0), total(freq, 1)
    
    # aantal totale, positieve en negatieve labels in de dataset
    D, Dpos, Dneg = len(lables), sum(lables), len(lables) - sum(lables)
    
    # de probability dat iets positief of negatief is
    PDpos, PDneg = Dpos / D, Dneg / D
    
    # bias berekenen van de dataset
    logprior = np.log(Dpos) - np.log(Dneg)
    
    # de log likelyhood van ieder woord berekenen
    log_like = {}
    for word, (Fpos, Fneg) in freq.items():
        log_like[word] = np.log(((Fpos + 1) / (Npos + v)) / ((Fneg + 1) / (Nneg + v)))
        
    return logprior, log_like

In [7]:
train_y = [1] * int(len(twitter_samples.strings("positive_tweets.json")) * .8) + [0] * int(len(twitter_samples.strings("negative_tweets.json")) * .8)
train_y = np.array(train_y)
len(train_y)

8000

In [8]:
logprior, logs = naive_bayes(freqs, trainset, train_y)
logprior

0.0

In [9]:
logs['great']

1.9768445311050868

niet precies hetzelfde maar komt dichtbij genoeg om er zeker van te zijn dat ik het goed doe

In [10]:
def predict(tweet, logprior, logs):
    return logprior + sum([logs[word] for word in clean(tweet)])

In [11]:
print('"I am happy" -> 2.14', end=" | ")
print(predict("I am happy", logprior, logs))
print('"I am bad" -> -1.31', end=" | ")
print(predict("I am bad", logprior, logs))
print('"this movie should have been great." -> 2.12', end=" | ")
print(predict("this movie should have been great.", logprior, logs))
print('"great" -> 2.13', end=" | ")
print(predict("great", logprior, logs))
print('"great great" -> 4.26', end=" | ")
print(predict("great great", logprior, logs))
print('"great great great" -> 6.39', end=" | ")
print(predict("great great great", logprior, logs))
print('"great great great great" -> 8.52', end=" | ")
print(predict("great great great great", logprior, logs))

"I am happy" -> 2.14 | 1.4434037184245863
"I am bad" -> -1.31 | -2.0194180233010863
"this movie should have been great." -> 2.12 | 1.9358916836415443
"great" -> 2.13 | 1.9768445311050868
"great great" -> 4.26 | 3.9536890622101737
"great great great" -> 6.39 | 5.9305335933152605
"great great great great" -> 8.52 | 7.907378124420347


ziet er goed uit

In [12]:
def accuracy(test_x, test_y, logprior, loglike):
    good = 0
    for index, tweet in enumerate(test_x):
        # print(index, tweet)
#         print(test_y[index], predict(tweet, logprior, loglike))
        if test_y[index] == (1 if predict(tweet, logprior, loglike) > 0 else 0):
            good += 1
    
    print(good, len(test_y))
    return good/len(test_y)

In [13]:
test_x = validatieset
test_y = np.array([1] * int(len(validatieset) / 2) + [0] * int(len(validatieset) / 2))
accuracy(test_x, test_y, logprior, logs)

2000 2000


1.0

100% lijkt raar, maar het lijkt allemaal goed te werken, voor de zekerheid heb ik het ook nog getest op de trainset en daar heeft hij een lagere acc op? Ik denk dat het gewoon een beetje geluk is dat het op 100 uitkomt.